In [30]:
from sklearn.metrics import *
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold
count_vect = CountVectorizer()

In [2]:
categories = ['talk.religion.misc', 'sci.crypt', 'misc.forsale', 'comp.sys.mac.hardware']

In [3]:
twenty_train = [fetch_20newsgroups(subset='train', categories=[i], shuffle=True, random_state=42) for i in categories]

In [4]:
twenty_test = [fetch_20newsgroups(subset='test', categories=[i], shuffle=True, random_state=0) for i in categories]

In [5]:
data_train = []

In [6]:
data_test = []

In [7]:
def add_data(initial, final):
    for i in initial:
        final += i.data

In [8]:
add_data(twenty_train, data_train)

In [9]:
add_data(twenty_test, data_test)

In [13]:
data_train[0]

'From: cj195@cleveland.Freenet.Edu (John W. Redelfs)\nSubject: Re: Disillusioned Protestant Finds Christ\nOrganization: Case Western Reserve University, Cleveland, OH (USA)\nLines: 22\nReply-To: cj195@cleveland.Freenet.Edu (John W. Redelfs)\nNNTP-Posting-Host: hela.ins.cwru.edu\n\n\nIn a previous article, tom@tredysvr.Tredydev.Unisys.COM (Tom Albrecht) says:\n\n>In article <1qb726$j9d@usenet.INS.CWRU.Edu< cj195@cleveland.Freenet.Edu (John W. Redelfs) writes:\n><\n><I learned that God loves his children who have never heard of him and has a\n><plan for redeeming them too, even those who have died without a knowledge\n><of Christ.  I learned that a man cannot frustrate justice by repenting on\n><his death bed because repentance is more than a feeling of remorse.  It\n><requires faith in Christ proven by following him, by keeping his\n><commandments.  Such cannot be accomplished on ones deathbed.\n>\n>So Jesus must have lied to the thief on the cross.\n\nParadise and salvation are not the

In [11]:
data_test[0]

'From: cfaehl@vesta.unm.edu (Chris Faehl)\nSubject: Re: Burden of Proof\nOrganization: University of New Mexico, Albuquerque\nLines: 35\nDistribution: world\nNNTP-Posting-Host: vesta.unm.edu\n\n>  hamilton@hydra.cs.gmr.com (Bill Hamilton) writes:\n> >cfaehl@vesta.unm.edu (Chris Faehl) writes:\n> >>ray@engr.LaTech.edu (Bill Ray) writes:\n\n> >********NOTE: FOLLOWUPS go to alt.atheism,talk.religion.misc,talk.origins!\n> >\n> >[deleted]\n> >[more deleted]\n> >If you say X statement and give it the authority of fact, I will respond\n> >by asking you why. You aren\'t obligated to say anything, but if your\n> >intent is to convince me that X statement is true, then yes, the burden\n> >of proof is upon you. \n> \n>[some interesting stuff, on the lines of the burden of proof belongs to God] \n> \n> >\n> >If you are merely giving testimony to your beliefs, then you are an egotist.\n> \n> Please excuse me if I missed an earlier part of this thread\n> in which Bill came across like an egotist. Wh

In [14]:
classes_train = []

In [15]:
classes_test = []

In [16]:
def add_classes(initial, final):
    for i in range(len(initial)):
        final += len(initial[i].data) * [i]

In [17]:
add_classes(twenty_train, classes_train)

In [21]:
add_classes(twenty_test, classes_test)

In [36]:
X_train = count_vect.fit_transform(data_train)
X_train.shape

(2135, 33456)

In [27]:
X_test = count_vect.transform(data_test)

In [28]:
def bestparams(model, grid, folds, data, classes):
    grid_search = GridSearchCV(model, param_grid=grid, cv=folds, scoring='f1_macro')
    grid_search.fit(data, classes) 
    return grid_search.best_score_, grid_search.best_params_, grid_search.best_estimator_

In [31]:
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=True, random_state=0)

In [32]:
models = [KNeighborsClassifier(), MultinomialNB(), LogisticRegression()]

In [33]:
grids = [{'n_neighbors': [5, 10, 15], 'leaf_size': [15, 30, 45]}, 
         {'alpha': [0.2, 0.5, 1, 2, 3],'fit_prior': [True, False]}, 
         {'penalty': ['l1', 'l2', 'elasticnet'], 'C': [1, 2, 3], 'max_iter': [100, 150, 200, 250]}]

In [34]:
chosenmodels = []

In [35]:
trainscores = []

In [37]:
 for i in range(3):
    best_score, best_params, best_estimator = bestparams(models[i], grids[i], folds, X_train.toarray(), classes_train)
    print('Best score is {}'.format(best_score))
    print('Best parameters are {}'.format(best_params))
    trainscores.append(best_score)
    chosenmodels.append(best_estimator)

Best score is 0.7554142497385083
Best parameters are {'leaf_size': 45, 'n_neighbors': 10}
Best score is 0.969398785771795
Best parameters are {'alpha': 0.2, 'fit_prior': True}


/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs 

/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got elasticnet penalty.

  FitFailedWarning)
/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarn

/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got elasticnet penalty.

  FitFailedWarning)
/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preproc

Best score is 0.9533371351532655
Best parameters are {'C': 2, 'max_iter': 100, 'penalty': 'l2'}


/Users/macbookpro/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
